In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)


examples = [
{
"movie": "Topgun",
"answer": """
Appropriate emojis include:
 "🛩️👨‍✈️🔥"
""",
},
{
"movie": "Godfather",
"answer": """
Appropriate emojis include:
"👨‍👨‍👦🔫🍝".
""",
},
{
"movie": "Interstellar",
"answer": """
Appropriate emojis include
"💫👨‍👧👩‍🚀".
""",
},
{
"movie": "Avatar: The Way of Water",
"answer": """
Appropriate emojis include
"👨‍👩‍👧‍👦🧜‍♂️🐋"
""",
},
{
"movie": "Frozen II",
"answer": """
Appropriate emojis include
"👭🧙‍♀️❄"
""",
},
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}? Answer with 3 emojis"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a film critic, you give answer with 3 emojis"),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "What do you know about {movie}?")
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context({"input": movie}, {"output": result.content})
    print(result)

In [2]:
invoke_chain("finding Dory")

Appropriate emojis include:
"🐠🐟🐋"content='Appropriate emojis include:\n"🐠🐟🐋"' response_metadata={'finish_reason': 'stop'}


In [3]:
invoke_chain("wreck it ralph")

Appropriate emojis include:
"🕹️👾👑"content='Appropriate emojis include:\n"🕹️👾👑"' response_metadata={'finish_reason': 'stop'}


In [4]:
invoke_chain("what was the previous movie I askedWhat was the previous movie I asked you about? Answer with a movie name, not an emoji")

The previous movie you asked about was "Finding Dory."content='The previous movie you asked about was "Finding Dory."' response_metadata={'finish_reason': 'stop'}
